<a href="https://colab.research.google.com/github/kyw061827/Yewon-KIM/blob/main/GEE_Climate_Aridity_TJK_Python_API(Xee).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
import geemap
import xarray as xr
!pip install xee
import xee

In [ ]:
ee.Authenticate()
ee.Initialize(
    project = 'gee-climate-aridity-tjk',
    opt_url = 'https://earthengine-highvolume.googleapis.com'
)

In [ ]:
#타지키스탄 구역 Square로 지정
map = geemap.Map(basemap = 'SATELLITE', ee_initialize=False)
map

In [ ]:
roi = map.draw_last_feature.geometry()
roi

In [ ]:
time_start = ee.Date('2005')
time_end = ee.Date('2025')
time_dif = time_end.difference(time_start, 'year').round()
time_list = ee.List.sequence(0, ee.Number(time_dif).subtract(1)).map(
    lambda x: time_start.advance(x, 'year')
)

def annual(date, col):
  start_date = ee.Date(date)
  end_date = start_date.advance(1, 'year')
  img_sum = col.filterDate(start_date, end_date).sum()
  return img_sum.set('system:time_start', start_date.millis())

gpm = (
    ee.ImageCollection("NASA/GPM_L3/IMERG_MONTHLY_V07")
    .filterDate(time_start, time_end)
    .select('precipitation')
    .map(lambda x: x.multiply(730).copyProperties(x, ['system:time_start']))
)

mod = (
    ee.ImageCollection("MODIS/061/MOD16A2GF")
    .filterDate(time_start, time_end)
    .select('PET')
    .map(lambda x: x.multiply(0.1).copyProperties(x, ['system:time_start']))
)

gpm_annual = ee.ImageCollection(time_list.map(lambda x: annual(x, gpm)))
mod_annual = ee.ImageCollection(time_list.map(lambda x: annual(x, mod)))

collection = gpm_annual.combine(mod_annual)

ds = xr.open_dataset(
    collection,
    engine ='ee',
    crs = 'EPSG:4326', #WGS84
    scale = 0.1,
    geometry = roi
)

In [ ]:
ds = ds.sortby('time') * 1

In [ ]:
ds

In [ ]:
aridity = ds.precipitation / ds.PET #PET: Potential evapotranspiration (증발산: 증산+발산, 토양 표면의 수분이 증발하고 식물 잎에서 증산 작용으로 물이 빠져나가는 과정을 합쳐서 부르는 용어로, 지구 표면의 물이 대기로 이동하는 핵심적인 수문 순환 과정 중 하나)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
aridity.plot.contourf(
    x = 'lon',
    y = 'lat',
    col = 'time',
    col_wrap = 5,
    robust = True, #Increase contrast for outputs (대비 상향 시켜주는 것)
    levels = 10, #범위를 몇 개로 나눌지에 대한 것
    cmap = 'turbo_r'
)

plt.savefig('aridity_map.png', dpi = 360, bbox_inches = 'tight')

In [ ]:
import numpy as np

In [ ]:
#Index of Aridity (adopted from UNEP)에 기반한 Classification
aridity_map = xr.where(aridity < 0.05, 1,
                       xr.where(
                           aridity < 0.2, 2,
                           xr.where(
                               aridity < 0.5, 3,
                               xr.where(
                                   aridity < 1, 4, np.nan
                               )
                           )
                       ))

In [ ]:
aridity_map.plot.contourf(
    x = 'lon',
    y = 'lat',
    col = 'time',
    col_wrap = 5,
    cmap = 'turbo_r'
)

plt.savefig('aridity_map_classified.png', dpi = 360, bbox_inches = 'tight')